In [6]:
import datasets
import pandas as pd
import os

In [7]:
# actionbility_mapping = {
#     "Unactionable": "1", 
#     "Borderline Actionable": "2", 
#     "Somewhat Actionable": "3", 
#     "Mostly Actionable": "4", 
#     "Highly Actionable": "5"
# }


def map_values(example):
    example["actionability"] = example["actionability"][0] if 'actionability' in example.keys() and example["actionability"] else None
    example["grounding_specificity"] = example["grounding_specificity"][0] if 'grounding_specificity' in example.keys() and example["grounding_specificity"] else None
    example['verifiability'] = example['verifiability'][0] if 'verifiability' in example.keys() and example['verifiability'] else None
    example['helpfulness'] = example['helpfulness'][0] if 'helpfulness' in example.keys() and example['helpfulness'] else None
    example['professional_tone'] = example['professional_tone'][0] if 'professional_tone' in example.keys() and example['professional_tone'] else None
    example['valid_point'] = example['valid_point'][0] if 'valid_point' in example.keys() and example['valid_point'] else None
    
    return example

In [8]:
BATCH = 5

dataset_folder = f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_{BATCH}'
human_annotations = []
## get all subdirectories in the dataset folder


with pd.ExcelWriter(f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_{BATCH}/human_annotation_{BATCH}.xlsx', engine="openpyxl") as writer:

    for human_data in os.listdir(dataset_folder):
        
        dataset_name = os.path.join(dataset_folder, human_data)

        ## if this is not a directory, skip
        if not os.path.isdir(dataset_name):
            continue

        ds = datasets.load_from_disk(dataset_name)

        column_names  = {
            'actionability.responses' : 'actionability', 
            'grounding_specificity.responses' : 'grounding_specificity',
            'verifiability.responses' : 'verifiability',
            'helpfulness.responses' : 'helpfulness',
            'professional_tone.responses' : 'professional_tone',
            'valid_point.responses' : 'valid_point',}
        ## delete columns that are not in the dataset
        column_names = {key: value for key, value in column_names.items() if key in ds.column_names}
        ds = ds.rename_columns(column_names)
        
        selected_columns = ['id','review_point', 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness', 'professional_tone', 'valid_point']
        ## select columns of they exist
        selected_columns = [column for column in selected_columns if column in ds.column_names]
        ds = ds.select_columns(selected_columns)

        ds = ds.map(map_values)

        df = ds.to_pandas()


        df.to_excel(writer, sheet_name=human_data, index=False)


In [9]:
df

,id,review_point,actionability,grounding_specificity,verifiability,helpfulness,professional_tone,valid_point
0,train_0,2. Missing some citations to set this in conte...,5,5,5,5,1,0
1,train_1,2. It would be better to compare with other se...,4,1,1,2,1,0
2,train_2,"2. Regarding the abstention process, it appear...",5,5,5,5,1,0
3,train_3,3. The comparison with Megatron is a little ov...,4,5,5,5,1,0
4,train_4,4) The analysis from line 128 to 149 is not co...,5,5,5,5,1,0
...,...,...,...,...,...,...,...,...
195,train_195,* The empirical analysis in Figure 3 is confus...,5,5,5,5,1,0
196,train_196,"3. Currently, this paper only considers binary...",1,2,3,2,1,0
197,train_197,- The recent related work CoCoOp [1] is not co...,5,5,5,4,1,0
198,train_198,- Fig. 1 can also be drawn better to show the ...,5,5,5,5,1,0


In [10]:
for annotation in human_annotations:
    annotator